# Import

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch
import torch.nn
import numpy as np 
import struct
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from facenet_model import *
from utils import *
import torch.optim as optim
import tqdm
import os

# Init Data

In [ ]:
PATH_RESULTS=""
CHECKPOINT_PATH=""

In [3]:
train_ds,val_ds=generate_datasets('./db_train.raw','./label_train.txt',0.7)
dataloader_t = torch.utils.data.DataLoader(train_ds,batch_size=32,shuffle=True,drop_last=False)
dataloader_v = torch.utils.data.DataLoader(val_ds,batch_size=32,shuffle=True,drop_last=False)

# Model

In [30]:
device= torch.device("cpu")
facenet=FaceNet(0.2,0.7,True).to(device)

In [31]:
optimizer = optim.Adam(facenet.parameters(), lr=3e-4, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5,verbose=True)
alpha1=0.2
alpha2=0.2
num_epochs=30
threshold=0.5
if CHECKPOINT_PATH=!"":
    checkpoint=torch.load(CHECKPOINT_PATH)
else:
    checkpoint=None

In [ ]:
training(num_epochs,facenet,optimizer,scheduler,dataloader_t,dataloader_v,device,alpha1,alpha2,threshold,PATH_RESULTS,checkpoint)

# Test

In [ ]:
test_ds=generate_datasets_test(path_images,path_label)
dataloader_test = torch.utils.data.DataLoader(test_ds,batch_size=32,shuffle=True,drop_last=False) #Dataloader train set

In [ ]:
device= torch.device("cuda:0")
checkpoint=torch.load("")
facenet=create_test_model(0.2,0.7,device,checkpoint)

In [ ]:
threshold=0.5
tp,tn,fp,fn=0,0,0,0
for i, dataj in enumerate(dataloader_test, 0): #Evaluation on validation set
    x=dataj[0].float().to(device)
    gt=dataj[1].float().to(device)
    y=facenet(x).view(-1)
    pred=y>threshold
    tp+=torch.sum(((pred)==gt)[torch.where((gt)==1)]).item()#TP
    fn+=torch.sum(((pred)!=gt)[torch.where((gt)==1)]).item()#FN
    tn+=torch.sum(((pred)==gt)[torch.where((gt)==0)]).item()#TN
    fp+=torch.sum(((pred)!=gt)[torch.where((gt)==0)]).item()#FP
far=fp/(tn+fp)
frr=fn/(tp+fn)
hter=0.5*(far+frr)
print("Hter : {} \n".format(hter))